<a href="https://colab.research.google.com/github/hursoo/Digital_History/blob/main/dh_gb_socialism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1_데이터 불러오기

## 1) 라이브러리, 구글 마운트

In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2) 파일 불러오기

In [ ]:
base_path = '/content/drive/MyDrive/Digital_History(230829)/'
file01 = '1_gb_sent.xlsx'
file02 = '2_r_ho.xlsx'
file03 = '3_ho_grid.xlsx'

In [ ]:
sent = pd.read_excel(base_path + file01)
r = pd.read_excel(base_path + file02)
ho = pd.read_excel(base_path + file03)

In [ ]:
sent[:3]

In [ ]:
r[:3]

In [ ]:
r1 = r.iloc[:,:4] # merge시 중복 피하기 위해 ho_no 제외
r1[:3]

In [ ]:
ho[:3]

## 3) db 병합하기

In [ ]:
temp1 = pd.merge(sent, r1, left_on = 'r_no', right_on = 'r_id', how = 'inner')
temp2 = pd.merge(temp1, ho, left_on = 'ho_no', right_on = 'ho_id', how = 'inner')
temp2[:3]

In [ ]:
# 필요한 열만 필요한 순서대로 바꾸기
df = temp2[['sent_id',  'sent_raw',  'sent_split',  'r_no',  'title',  'writer',  'w_new',  'ho_no',  'year',  'month',  'grid']]
print(df.shape)
df[:3]

In [ ]:
df.to_excel(base_path + 'gb_corpus.xlsx')

# 2_논조를 반영하는 특성 추출

## 1) 문서-단어 행렬(dtm) 산출 함수

In [ ]:
def get_dtm(df, col_name, stopw, rank_n): # rank_n : 고빈도 단어 n 순위까지
    # 단어 종류 모두 벡터화. 2음절 이상
    tv = TfidfVectorizer(stop_words=stopw, norm=None)
    dtm = tv.fit_transform(df[col_name])

    # df 형태로 표시
    dtm_df = pd.DataFrame(dtm.toarray(), columns=tv.get_feature_names_out(), index=df.index)

    highword_list = dtm_df.sum().sort_values(ascending=False)[:rank_n].index.to_list()
    feature_df = dtm_df[highword_list] # 열 순서는 tfidf값이 높은 것부터 낮은 순으로 정렬
    return feature_df

## 2) 특성 및 특성벡터
- tfidf 고빈도 50위 단어

In [ ]:
stopword_3 = ['문제', '금일', '관계'] # 제외할 단어

dtm50_df = get_dtm(df, 'sent_split', stopword_3, 50)
dtm50_df

#3_시기 구분하기
- 시기 구분 : 논조 변화를 관찰하는 마디

## 1) dtm50 + 구간(grid) = gtm50 산출하기

In [ ]:
def transform_to_gtm(df, grid_col, dtm_df): # 인자 - df, 구간 정보 열, dtm50_df
    # 구간정보만 df로 추출
    grid_df = df[[grid_col]]
    # 구간 정보 결합하고, 구간을 index로 만듦
    temp_dtm = pd.concat([dtm_df, grid_df], axis=1)
    grid_dtm = temp_dtm.set_index(grid_col)
    # 구간별 평균
    gtm = grid_dtm.groupby(grid_col).mean()
    return gtm

In [ ]:
# 함수 실행하여 gtm 변수에 할당
gtm_df = transform_to_gtm(df, 'grid', dtm50_df)
gtm_df

## 2) 구간별 상관계수 산출하기

In [ ]:
def make_corr(gtm):
    tgm = gtm.T
    result = tgm.corr()
    return result

In [ ]:
gg_corr = make_corr(gtm_df)
gg_corr

## 3) 히트맵 시각화로 시기구분 하기

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def draw_heatmap(df):
    df = df.copy()
    plt.figure(figsize=(10,4))
    sns.heatmap(data = df, annot=True,
    fmt = '.3f', linewidths=.5, cmap='Blues')

In [ ]:
draw_heatmap(gg_corr)

- 모호한 부분의 시기 구분  
 - 해당 구간이 좌우와 가진 상관계수를 비교  
 - 이 때 비교 범위는 왼쪽(및 오른쪽) 가상 시기에 포함되는 구간들의 상관계수를 평균한 값  
 =>   
    1p : 01hf - 05hf  
    2p : 06hf - 09hf  
    3p : 10hf - 12hf

## 4) 시기별 코퍼스 산출

In [ ]:
prd_1 = ['01hf', '02hf', '03hf', '04hf', '05hf']
prd_2 = ['06hf', '07hf', '08hf', '09hf']
prd_3 = ['10hf', '11hf', '12hf']

In [ ]:
df = df.copy()
df['period'] = '' # df에 새 열('period')을 생성
df.loc[df['grid'].isin(prd_1), 'period'] = '1p'
df.loc[df['grid'].isin(prd_2), 'period'] = '2p'
df.loc[df['grid'].isin(prd_3), 'period'] = '3p'
df

In [ ]:
# 시기구분 정보 반영 검증
df.groupby(['period', 'grid'])['sent_id'].count()

In [ ]:
# 시기별 코퍼스

df_1p = df[df['period'] == '1p']
df_2p = df[df['period'] == '2p']
df_3p = df[df['period'] == '3p']

In [ ]:
print(df_1p.shape)
print(df_2p.shape)
print(df_3p.shape)
print(df_1p.shape[0] + df_2p.shape[0] + df_3p.shape[0])

# 4_시기별 연결망 계수, 지표 산출

## 1) 시기별 코사인유사도 계수

In [ ]:
# 코사인유사도 산출 함수

def get_cossim(dtm_df):
    tdf_n = dtm_df.columns.tolist()
    tdm = dtm_df.T
    cossim = cosine_similarity(tdm, tdm)
    cossim_df = pd.DataFrame(cossim, columns=tdf_n, index=tdf_n)
    return cossim_df

In [ ]:
# 시기별 dtm 산출

stopword_3 = ['문제', '금일', '관계'] # 제외할 단어
dtm50_df_1p = get_dtm(df_1p, 'sent_split', stopword_3, 50)
dtm50_df_2p = get_dtm(df_2p, 'sent_split', stopword_3, 50)
dtm50_df_3p = get_dtm(df_3p, 'sent_split', stopword_3, 50)

In [ ]:
print(dtm50_df_1p.shape)
print(dtm50_df_2p.shape)
print(dtm50_df_3p.shape)

In [ ]:
# 시기별 코사인 유사도 산출 함수 실행

cossim_1p = get_cossim(dtm50_df_1p)
cossim_2p = get_cossim(dtm50_df_2p)
cossim_3p = get_cossim(dtm50_df_3p)

In [ ]:
print(cossim_1p.iloc[:3,:5])
print(cossim_2p.iloc[:3,:5])
print(cossim_3p.iloc[:3,:5])

## 2) wnet.exe 입력값 작성

In [ ]:
# 코사인유사도 df를 그대로 텍스트 파일(tsv)로 저장 (인덱스, 헤더 모두 출력하도록 해야 고빈도 50개 단어가 출력됨)

cossim_1p.to_csv(base_path + 'result/12_wnet_input_1p.txt', index=True, header=True, sep='\t')
cossim_2p.to_csv(base_path + 'result/12_wnet_input_2p.txt', index=True, header=True, sep='\t')
cossim_3p.to_csv(base_path + 'result/12_wnet_input_3p.txt', index=True, header=True, sep='\t')

## 3) nc.exe 입력값 작성

In [ ]:
# nc(지역중심성) 산출 위한 입력값 만들기 - "동일단어-동일단어" 제거. 2,500행 -> 2,450행

def cossim_to_ncinput(df):
    # df : 코사인유사도 df
    df1 = pd.DataFrame(df.unstack())
    not_same_index = [(m,n) for m, n in df1.index if (m != n)]
    not_same_df = df1.loc[not_same_index]
    return not_same_df

In [ ]:
# nc(지역중심성) 산출 위한 입력값 만들기 - "동일단어-동일단어" 제거. 2,500행 -> 2,450행
# 텍스트 파일(tsv)로 저장

cossim_to_ncinput(cossim_1p).to_csv(base_path + 'result/12_ncinput_1p.txt', header=False, sep='\t')
cossim_to_ncinput(cossim_2p).to_csv(base_path + 'result/12_ncinput_2p.txt', header=False, sep='\t')
cossim_to_ncinput(cossim_3p).to_csv(base_path + 'result/12_ncinput_3p.txt', header=False, sep='\t')

## 4) 계수,지표 산출(윈도우 커맨드)
- wnet.exe
- nc.exe
- 결과 파일을 다시 구글드라이브의 data 폴더에 업로드

# 5_노드엑셀 입력값 산출

## 1) 중복순서쌍 제거

In [ ]:
# PFNet 파일 불러오기
pfnet_1p = pd.read_csv(base_path + 'data/PFNet-12_wnet_input_1p.txt', sep='\t')
pfnet_2p = pd.read_csv(base_path + 'data/PFNet-12_wnet_input_2p.txt', sep='\t')
pfnet_3p = pd.read_csv(base_path + 'data/PFNet-12_wnet_input_3p.txt', sep='\t')

In [ ]:
pfnet_1p

In [ ]:
# 순서만 다르고 요소는 같은 순서쌍 중 하나를 제거하는 함수

def remove_same_pairwords(df):  # df : pfnet df

    # df에서 단어 순서쌍만 리스트로 추출
    df_num = df.index.tolist()
    ww_list = []
    for i in df_num:
        ww_list.append(df.iloc[i,:2].tolist())
    ww_list

    # 순서쌍의 원소가 동일 단어인 경우 추출
    dupli_num=[]
    for i, (x, y) in enumerate(ww_list):
        if [y, x] in ww_list:
            dupli_num.append(i)
    dupli_num

    # 순서쌍이 동일한 것 중 하나만 남겨 df 출력
    if len(dupli_num) > 0: # 중복 순서쌍 있을 경우
        only_one = [k for k in df_num if k not in dupli_num] # 중복 안되는 index 일단 추출
        del_dupli = only_one + [dupli_num[0]] # 중복되는 것 중 하나만 index 추가
        del_dupli.sort() # index 정렬
        result = df.iloc[del_dupli] # index로 df 불러옴
        result1 = result.reset_index(drop=True) # index를 새로 고침
        return result1
    else:  # 중복되는 것 없으면 그대로 df 출력
        return df

In [ ]:
pfnet_1p_df = remove_same_pairwords(pfnet_1p)
pfnet_2p_df = remove_same_pairwords(pfnet_2p)
pfnet_3p_df = remove_same_pairwords(pfnet_3p)

In [ ]:
pfnet_1p_df[:3]

## 2) 코사인유사도 등급화

In [ ]:
# weight 값을 범주형으로 (5등분) for 링크(엣지) 시각화

def transform_wgt_to_grade(df, col_name, grade_no): # col_name : '[Weight]'
    df_c = df.copy() # 입력 df 자체의 weight가 변화하는 것 방지용

    # 등급(grade) 간 격차를 0.5로 설정
    df_c[col_name] = pd.qcut(df_c[col_name], q=grade_no, labels=list(np.arange(1,1+grade_no/2,0.5)))

    # 열 이름을 알기 쉽게 변경
    df1 = df_c.rename(columns = {'[Node1]' : 'node1',
                                '[Node2]' : 'node2',
                                '[Weight]': 'weight'})
    return df1

In [ ]:
pfnet_1p_df_simple_wgt = transform_wgt_to_grade(pfnet_1p_df, '[Weight]', 5)
pfnet_2p_df_simple_wgt = transform_wgt_to_grade(pfnet_2p_df, '[Weight]', 5)
pfnet_3p_df_simple_wgt = transform_wgt_to_grade(pfnet_3p_df, '[Weight]', 5)

In [ ]:
# 구분 검증
pfnet_1p_df_simple_wgt['weight'].value_counts().to_frame() # weight 값을 값별로 센 결과(시리즈)를 데이터프레임으로 변경

In [ ]:
# 엑셀 파일로 저장하기

pfnet_1p_df_simple_wgt.to_excel(base_path + 'result/12_pfnet_1p_nodexl.xlsx')
pfnet_2p_df_simple_wgt.to_excel(base_path + 'result/12_pfnet_2p_nodexl.xlsx')
pfnet_3p_df_simple_wgt.to_excel(base_path + 'result/12_pfnet_3p_nodexl.xlsx')

## 3) 군집 및 중심성 정보 정리

In [ ]:
# PNNC 파일 불러오기
pnnc_1p = pd.read_csv(base_path + 'data/PNNC-12_wnet_input_1p.txt', sep='\t')
pnnc_2p = pd.read_csv(base_path + 'data/PNNC-12_wnet_input_2p.txt', sep='\t')
pnnc_3p = pd.read_csv(base_path + 'data/PNNC-12_wnet_input_3p.txt', sep='\t')

# WCENT 파일 불러오기
wcent_1p = pd.read_csv(base_path + 'data/WCENT-12_wnet_input_1p.txt', sep='\t')
wcent_2p = pd.read_csv(base_path + 'data/WCENT-12_wnet_input_2p.txt', sep='\t')
wcent_3p = pd.read_csv(base_path + 'data/WCENT-12_wnet_input_3p.txt', sep='\t')

# NC 파일 불러오기
nc_1p = pd.read_csv(base_path + 'data/nc_2.0_12_ncinput_1p.txt', sep='\t')
nc_2p = pd.read_csv(base_path + 'data/nc_2.0_12_ncinput_2p.txt', sep='\t')
nc_3p = pd.read_csv(base_path + 'data/nc_2.0_12_ncinput_3p.txt', sep='\t')

In [ ]:
print(pnnc_1p[:3])
print(wcent_2p[:3])
print(nc_3p[:3])

In [ ]:
# WNET0.4 결과물 중에서 꼭 필요한 세 지표만 산출하기

def get_group_ctrlity(pnnc, wcent, nc):
    group = pnnc
    global_ctrlity = wcent
    local_ctrlity = nc
    a = group.merge(global_ctrlity[['NODE', 'rTBC(0~1)']], left_on='[Item]', right_on='NODE', how='inner')
    b = a.merge(local_ctrlity, left_on='[Item]', right_on='[NODE]', how='inner')
    b = b.drop(['[SN]', 'NODE', '[NODE]'], errors='ignore', axis=1)
    return b

In [ ]:
# 그룹 및 중심성 지표 산출 함수 실행

group_ctrlity_1p = get_group_ctrlity(pnnc_1p, wcent_1p, nc_1p)
group_ctrlity_2p = get_group_ctrlity(pnnc_2p, wcent_2p, nc_2p)
group_ctrlity_3p = get_group_ctrlity(pnnc_3p, wcent_3p, nc_3p)

In [ ]:
print(group_ctrlity_1p[:3])
print(group_ctrlity_2p[:3])
print(group_ctrlity_3p[:3])

In [ ]:
# 엑셀로 저장하기

group_ctrlity_1p.to_excel(base_path + 'result/12_group_ctrlity_1p_nodexl.xlsx')
group_ctrlity_2p.to_excel(base_path + 'result/12_group_ctrlity_2p_nodexl.xlsx')
group_ctrlity_3p.to_excel(base_path + 'result/12_group_ctrlity_3p_nodexl.xlsx')

# The End of Note